In [1]:
import os, sys, csv, json, re
import numpy as np
import pandas as pd

In [2]:
work_folder = 'SF_100'
scene_folder = 'scene-json'
blue_nodes = {}
with open(os.path.join(scene_folder, 'meta', 'blue_nodes_97_new.json'), 'r') as f:
    cityffd_meta = json.load(f)
    for node in cityffd_meta:
        node_id = node['id']
        blue_nodes[node_id] = node['properties']['centroid']

In [3]:
len(blue_nodes)

3303

In [4]:
import math
mapping_json = {}
mapping_json['features'] = []
with open(os.path.join(scene_folder, 'meta', 'sf_dt_98_red_nodes_data_exchanges.json'), 'r') as f:
    eplus_meta = json.load(f)
    for bldg in eplus_meta['features']:
        bldg_id = bldg['id']
        mapping_json_bldg = {}
        mapping_json_bldg['id'] = bldg_id
        mapping_json_bldg['properties'] = {}
        mapping_json_bldg['properties']['objects'] = []
        for surf in bldg['properties']['objects']:
            distance = -1
            blue_node = None
            surf_id = surf['id']
            mapping_json_surf = {}
            red_coord = surf['properties']['centroid']
            for blue_node_id in blue_nodes:
                blue_coord = blue_nodes[blue_node_id]
                curr_dis = math.sqrt((blue_coord[0] - red_coord[0]) ** 2 +
                                     (blue_coord[1] - red_coord[1]) ** 2 +
                                     (blue_coord[2] - red_coord[2]) ** 2)
                if distance == -1 or curr_dis < distance:
                    distance = curr_dis
                    blue_node = blue_node_id
#             print(blue_node, distance)
            mapping_json_surf['id'] = surf_id
            mapping_json_surf['properties'] = {}
            mapping_json_surf['properties']['cityffd_id'] = blue_node
            mapping_json_bldg['properties']['objects'].append(mapping_json_surf)
            
        mapping_json['features'].append(mapping_json_bldg)
    file = open(os.path.join(scene_folder, 'meta', 'sf_dt_97_mapping.json'), 'w')
    file.write(json.dumps(mapping_json, indent=2))
    file.close()